# Text classification of Movie Reviews

Classifies movie reviews as positive or negative using the text of the review.

Following this [Tensorflow tutorial](https://www.tensorflow.org/beta/tutorials/keras/basic_text_classification_with_tfhub).

## Import Packages

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Tensorflow and Keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Helper libraries

# Visualization libraries


## Import the Data

### Load Training and Testing Sets

In [5]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)    # Structures dataset as tuple (features, label)

W0817 13:04:56.431564 4691973568 deprecation.py:323] From /Users/josh/anaconda3/lib/python3.7/site-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/josh/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


### Explore the Data 

In [22]:
num_reviews = 10    # Number of reviews to review
train_examples_batch, train_labels_batch = next(iter(train_data.batch(num_reviews)))
for i in range(num_reviews):
    print( 'Rating: {0:d}\nSentence: {1}\n'.format(train_labels_batch[i], train_examples_batch[i]) )


Rating: 1
Sentence: b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novel

### Prepare Data

In [23]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=907, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

## Build Model

In [24]:
model = tf.keras.Sequential()
model.add(hub_layer)                                         # Input layer as vocab encoding
model.add(tf.keras.layers.Dense(16, activation='relu'))      # Single small hidden layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))    # One sigmoid node since label is binary

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Select Loss Function and Optimizer

In [25]:
model.compile(optimizer='adam',              # Optimizer favored over standard gradient decent
              loss='binary_crossentropy',    # Favored loss function for single probabilistic output
              metrics=['accuracy'])          # Standard metric for performance

## Train the Model

In [29]:
history = model.fit(train_data.shuffle(10000, reshuffle_each_iteration=True) # Suffle the dataset at the start of each iteration
                              .batch(512),                         # Perform updates for each batch of 512 samples
                    epochs=20,                                     # Perform 20 iterations over all dataset
                    validation_data=validation_data.batch(512),    # Test performance with validation set
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 99ms/step - loss: 0.3293 - accuracy: 0.8741 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 3s 98ms/step - loss: 0.3099 - accuracy: 0.8797 - val_loss: 0.3627 - val_accuracy: 0.8454
Epoch 3/20
30/30 [==============================] - 3s 94ms/step - loss: 0.2916 - accuracy: 0.8866 - val_loss: 0.3511 - val_accuracy: 0.8507
Epoch 4/20
30/30 [==============================] - 3s 94ms/step - loss: 0.2746 - accuracy: 0.8945 - val_loss: 0.3410 - val_accuracy: 0.8557
Epoch 5/20
30/30 [==============================] - 3s 99ms/step - loss: 0.2588 - accuracy: 0.9005 - val_loss: 0.3323 - val_accuracy: 0.8597
Epoch 6/20
30/30 [==============================] - 3s 97ms/step - loss: 0.2440 - accuracy: 0.9052 - val_loss: 0.3249 - val_accuracy: 0.8634
Epoch 7/20
30/30 [==============================] - 3s 101ms/step - loss: 0.2302 - accuracy: 0.9121 - val_loss: 0.3189 - val_accuracy: 0.8677
Epoc

## Evaluate the Model

In [30]:
results = model.evaluate(test_data.batch(512), verbose=1)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

     49/Unknown - 2s 43ms/step - loss: 0.3411 - accuracy: 0.8623loss: 0.341
accuracy: 0.862
